# Travel Comment Classification
### YouTube Travel Buzz NLP Pipeline – Module 1

Binary classification model to identify travel-related YouTube comments.


### 0. Execution Note

This notebook contains **actual training, evaluation, and inference outputs**
generated during the project period.

Due to the use of private local paths and time-bound compute environments:
- File paths were anonymized
- Authentication-related steps were removed
- The notebook is **not intended for re-execution**

All model logic, metrics, and outputs remain unchanged.


### 1. Environment Setup

Required libraries for training and evaluation.


In [ ]:
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


### 2. Training Data Loading

- Input: CSV file containing `comment` and `label`
- Labels: 0 (Non-travel), 1 (Travel-related)


In [ ]:
file_path = "data/travel_comment_train.csv"  # anonymized path

# NOTE:
# Original data was loaded from a local private path.
# Path anonymized for public GitHub release.
df = pd.read_csv(file_path)


### 3. Dataset Construction

Convert pandas DataFrame to Hugging Face Dataset and split into train/test.


In [ ]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)


### 4. Model & Tokenizer Initialization

Base model: `monologg/koelectra-small-discriminator`


In [ ]:
model_name = "monologg/koelectra-small-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2
)


### 5. Tokenization


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["comment"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)


### 6. Evaluation Metrics


In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


### 7. Training Configuration


In [ ]:
training_args = TrainingArguments(
    output_dir="./koelectra_results",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./koelectra_logs",
    logging_steps=10,
)


### 8. Model Training


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

trainer.train()


### 9. Model Saving


In [ ]:
# Model was saved locally during the project period.
# Path omitted for public release.

final_save_path = "./models/travel_comment_classifier"
trainer.save_model(final_save_path)
tokenizer.save_pretrained(final_save_path)

print(f"✅ 최종 모델 저장 완료: {final_save_path}")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 403/403 [00:00<00:00, 8526.17 examples/s]


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.624800,0.591907,0.853598,0.875264
2,0.486500,0.413524,0.940447,0.943128
3,0.373100,0.323995,0.937965,0.942263
4,0.289400,0.266958,0.947891,0.950355
5,0.240800,0.253103,0.947891,0.950355


✅ 최종 모델 저장 완료: C:\Users\rhksd\Desktop\Data\항공\travel_comment_classifier_koelectra_v2000


### 10. Single Comment Inference Setup


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_path = "model/travel_comment_classifier"  

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

def predict_comment(text):
    if not isinstance(text, str) or not text.strip():
        return -1, 0.0

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = round(probs[0][pred].item(), 4)

    return pred, confidence


### 11. Batch Inference on YouTube Comments

Apply the classifier to real YouTube comments and analyze confidence.


In [ ]:
from pathlib import Path

file_path = Path("data/youtube_comments.csv")  # anonymized
df = pd.read_csv(file_path)

df[["predict_label", "confidence"]] = df["comment"].astype(str).apply(
    lambda x: pd.Series(predict_comment(x))
)

print("\n[예측 결과 미리보기]")
print(df.head())


### 12. Prediction Summary & Human-in-the-Loop Signals


In [ ]:
label_counts = df["predict_label"].value_counts().sort_index()
label_counts

df["confidence"].mean()

ambiguous = df[(df["confidence"] >= 0.45) & (df["confidence"] <= 0.6)]
len(ambiguous)


In [ ]:
label_counts = df["predict_label"].value_counts().sort_index()
print("\n[예측 라벨 분포]")
for label, count in label_counts.items():
    name = {0: "기타", 1: "여행 관련", -1: "예측 불가"}.get(label, "기타")
    print(f"- {name}: {count}개")

mean_confidence = df["confidence"].mean()
print("\n[전체 평균 확신도]")
print(f"- 평균 확신도: {mean_confidence:.4f}")

ambiguous = df[(df["confidence"] >= 0.45) & (df["confidence"] <= 0.6)]
print("\n[확신도 0.45 ~ 0.6 구간]")
print(f"- 애매한 예측 수: {len(ambiguous)}개")

print("\n[애매한 예측 샘플 예시 (상위 5개)]")
print(ambiguous[["comment", "predict_label", "confidence"]].head())



[예측 결과 미리보기]
       videoId                                            comment  likeCount  \
0  -1EnPvTX5UY  자세한 영상 잘봤습니다! 베트남은 첨이라 버기 나 벨보이 부를때마다 팁주나요? 혹시...          1   
1  -1EnPvTX5UY  @@kennybang6252 감사합니다! 팁은 개인의 선택인것같습니다. 저는 짐이 ...          0   
2  -1EnPvTX5UY                                    배달전화 올려주실수 있나요?          0   
3  -1EnPvTX5UY        @@sunkim4877 배달K라는 어플을 다운받으시면 배민처럼 이용가능합니다.          0   
4  -1EnPvTX5UY              영상 잘 봤어요! 혹시 호텔에서 나쨩시내 가는 셔틀버스는 없었나요?          0   

            publishedAt  predict_label  confidence  
0  2025-03-08T03:41:35Z            1.0      0.7823  
1  2025-03-08T04:12:28Z            1.0      0.7760  
2  2025-01-21T10:08:03Z            0.0      0.8191  
3  2025-01-21T10:29:32Z            1.0      0.7633  
4  2024-08-15T13:13:02Z            1.0      0.8194  

[저장 완료] 예측 결과가 다음 경로에 저장되었습니다:
C:\Users\rhksd\Desktop\Data\항공\유튜브 22 23 24\nhatrang_comments_2022_2024_full_labeled.csv

[예측 라벨 분포]
- 기타: 51989개
- 여행 관련: 55862개

[전체 평균 확신도]
- 평균 확신도: 0.

### 13. Interpretation of Inference Results

The inference results indicate that the travel comment classifier
successfully distinguishes travel-related discourse from general
engagement noise in real-world YouTube comments.

Key observations from the output above include:

- A balanced distribution between travel-related and non-travel comments,
  suggesting that the model is not biased toward over-classification.
- A high average confidence score, indicating stable and consistent
  predictions across a large-scale dataset.
- The presence of ambiguous cases (confidence 0.45–0.60), which reflects
  realistic linguistic overlap in conversational comments and validates
  the necessity of a human-in-the-loop review strategy.

These filtered travel-related comments are subsequently passed to the
downstream sentiment analysis and demand signal interpretation modules
within the YouTube Travel Buzz NLP Pipeline.
